<a href="https://colab.research.google.com/github/napoler/tkitAutoMark-pytorch/blob/master/tkitAutoMark%E7%A4%BA%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tkitAutoMark==0.0.0.1.4

     |████████████████████████████████| 808kB 2.8MB/s 
     |████████████████████████████████| 2.3MB 13.6MB/s 
     |████████████████████████████████| 829kB 19.2MB/s 
     |████████████████████████████████| 112kB 23.2MB/s 
     |████████████████████████████████| 645kB 19.2MB/s 
     |████████████████████████████████| 10.6MB 22.3MB/s 
     |████████████████████████████████| 276kB 36.6MB/s 
     |████████████████████████████████| 3.3MB 32.3MB/s 
     |████████████████████████████████| 901kB 32.8MB/s 
     |████████████████████████████████| 1.3MB 35.3MB/s 
     |████████████████████████████████| 143kB 41.4MB/s 
     |████████████████████████████████| 296kB 19.7MB/s 
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-cp37-none-any.whl size=2905 sha256=251c2d98ab3678b87e69d6f51681851611a9feffc9a8c06e19412ca59a007fa8
  Stored in directory: /root/.cache/pip/wheels/cd/f8/93/25b60e319a481e8f324dcb1871aff818eb0c8143ed20b732b4
  Created wheel for future: fil

In [2]:
from tkitAutoMark import AutoMark


tModel=AutoMark(learning_rate=5e-5,num_labels=5,dim=128)
# tModel

In [3]:
# help(AutoMark)

# 示例

In [4]:
# !wget https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip
# !unzip cluener_public.zip

In [5]:
# !pip install tkitJson
!pip install datasets transformers

     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 245kB 12.4MB/s 


In [6]:
!git clone https://github.com/napoler/NER-BERT-pytorch

Cloning into 'NER-BERT-pytorch'...
remote: Enumerating objects: 80, done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 80
Unpacking objects: 100% (80/80), done.


In [7]:
labels={}
with open("/content/NER-BERT-pytorch/data/msra/tags.txt",'r') as f:
  for i,line in enumerate(f):
    tag=line.replace("\n",'')
    if len(tag)>0:
      labels[tag]=i

In [8]:
# labels
labels

{'B-LOC': 6,
 'B-ORG': 1,
 'B-PER': 3,
 'I-LOC': 4,
 'I-ORG': 5,
 'I-PER': 2,
 'O': 0}

In [9]:
# Process the dataset - tokenize the context texts (using a tokenizer from the 🤗Transformers library)
from transformers import BertTokenizer
from tqdm.auto import tqdm
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained("clue/roberta_chinese_clue_tiny")
tokenizer
# tokenized_dataset = squad_dataset.map(lambda x: tokenizer(x['tokens']), batched=True)



def Processing(file="/content/NER-BERT-pytorch/data/msra/msra_test_bio",labels={}):
  test_tags=[]
  test_data=[]
  # tags=[]
  # data=[]
  with open(file,'r') as f:
    for line in tqdm(f):
      # print()
      one=line.split("\t")
      if len(one)==2:
        # print(one)


        tkout = tokenizer(one[0])
        tkdata=tkout["input_ids"][1:-1]
        test_data=test_data+tkdata

        tag=one[1].replace("\n",'')
        if len(tag)>0:
          test_tags=test_tags+[labels[tag]]*len(tkdata)
      else:
          test_tags=test_tags+[labels["O"]]
          test_data=test_data+[101]

      assert len(test_data)==len(test_tags)
      # else:
      #   if len(data)>max:
      #     tags=[]
      #     data=[]
      #   else:
      #     # data=data+[0]*(128-len(data))
      #     # tags=tags+[labels["O"]]*(128-len(data))
      #     test_data.append(data)
      #     test_tags.append(tags)
      #     tags=[]
      #     data=[]
    # break
  padNum=(len(test_data)//4096+1)*4096-len(test_data)
  test_data=test_data+padNum*[0]
  test_tags=test_tags+padNum*[labels["O"]]
  return test_data,test_tags

test_data,test_tags=Processing("/content/NER-BERT-pytorch/data/msra/msra_test_bio",labels)
train_data,train_tags=Processing("/content/NER-BERT-pytorch/data/msra/msra_train_bio",labels)

In [10]:
# test_data[:1],test_tags[:1]
# tokenizer("[CLS]")
# len(test_data)//4096+1

In [11]:
import torch
from torch.utils.data import DataLoader,TensorDataset

# t=torch.Tensor([test_data])
# t.size()
trainDataset=TensorDataset(torch.Tensor([train_data]).view(-1,4096),torch.Tensor([train_tags]).view(-1,4096))
testDataset=TensorDataset(torch.Tensor([test_data]).view(-1,4096),torch.Tensor([test_tags]).view(-1,4096))

In [12]:
# testDataset[:2][1].long()

In [13]:
import pickle
with open("train.pkt",'wb') as f:
  pickle.dump(trainDataset,f)
with open("test.pkt",'wb') as f:
  pickle.dump(testDataset,f)

In [14]:
trainDataset=TensorDataset(torch.Tensor([train_data]).view(-1,128).long(),torch.Tensor([train_tags]).view(-1,128).long())
testDataset=TensorDataset(torch.Tensor([test_data]).view(-1,128).long(),torch.Tensor([test_tags]).view(-1,128).long())


train_loader = torch.utils.data.DataLoader(trainDataset,
                                            batch_size=30,
                                            shuffle=True,
                                            num_workers=2)
test_loader = torch.utils.data.DataLoader(testDataset,
                                            batch_size=30,
    #                                         shuffle=True,
                                            num_workers=2)

In [26]:
# /kaggle/input/reformerchinesemodel/epoch4step21209.ckpt
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
# 自动停止
# https://pytorch-lightning.readthedocs.io/en/1.2.1/common/early_stopping.html
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.callbacks import LearningRateMonitor

import pytorch_lightning as pl

num_labels=len(labels)
tModel=AutoMark(dim=128,
                    max_seq_len=512,
                    num_labels=num_labels) 



trainer = pl.Trainer(
    gpus=1,
# #     min_epochs=1,
    precision=16,amp_level='O2',
#         checkpoint_callback=checkpoint_callback,
#         resume_from_checkpoint="../input/nernotebook61fed2341fmodel/LitAutoMark-out.ckpt",
    # auto_select_gpus=True,
    # callbacks=[lr_monitor,early_stop_callback],
    deterministic=True,
#         auto_scale_batch_size='binsearch',
#         auto_lr_find=True,
#         max_epochs=wandb.config.epochs,
    max_epochs=1,
    # logger=wandb_logger,
#         accumulate_grad_batches=wandb.config.accumulate_grad_batches
    accumulate_grad_batches=1
)
# train
trainer.fit(tModel,train_loader,test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | model      | PerformerLM | 3.4 M 
1 | classifier | MLP         | 1.0 M 
2 | crf        | CRF         | 63    
3 | f1_loss    | F1_Loss_crf | 0     
-------------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
17.883    Total estimated model params size (MB)


Epoch     0: adjusting learning rate of group 0 to 1.0000e-04.


Epoch     1: adjusting learning rate of group 0 to 9.0451e-05.



In [29]:
for it in test_loader:
  print(it)

  out=tModel(it[0],return_pred=True)
  print(out[0])
  break

[tensor([[ 604,  914,  604,  ..., 1049, 1040, 1841],
        [5402, 4666,  910,  ...,  916, 2179, 3904],
        [4086,  729, 5822,  ..., 5181, 2653,  544],
        ...,
        [1273, 1105, 5181,  ...,  544, 1828,  908],
        [ 544,  939, 2549,  ...,  714,  545,  101],
        [ 611, 3083, 6294,  ..., 3904, 6294, 1185]]), tensor([[1, 5, 5,  ..., 0, 0, 0],
        [0, 1, 5,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])]
tensor([[6, 0, 6,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


# 示例


In [16]:
!pip install datasets transformers

In [17]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

# Print all the available datasets
print(list_datasets())

['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect', 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset', 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato', 'babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli', 'bc2gm_corpus', 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac', 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'caner', 'capes', 'catalonia_independence', 'cawac', 'cbt', 'c

In [18]:
# Load a dataset and print the first example in the training set
weibo_dataset = load_dataset('weibo_ner')
print(weibo_dataset['train'][0])

Using custom data configuration default


Dataset weibo_ner_corpus downloaded and prepared to /root/.cache/huggingface/datasets/weibo_ner_corpus/default/1.0.0/4a93f6e0f1024e5dd7a7ea6c604e5a5455bb8ff6cf87ff76dcd057fe99a0f62a. Subsequent calls will reuse this data.
{'id': '0', 'ner_tags': [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16], 'tokens': ['科0', '技1', '全0', '方1', '位2', '资0', '讯1', '智0', '能1', '，0', '快0', '捷1', '的0', '汽0', '车1', '生0', '活1', '需0', '要1', '有0', '三1', '屏2', '一0', '云1', '爱2', '你0']}


In [19]:
# List all the available metrics
# 列出所有的评价指标
print(list_metrics())

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'comet', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'indic_glue', 'matthews_correlation', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'wer', 'xnli']


In [20]:
# Load a metric
# squad_metric = load_metric('weibo_ner')

In [21]:
# Process the dataset - add a column with the length of the context texts
dataset_with_length = weibo_dataset.map(lambda x: {"length": len(x["tokens"])})

In [22]:
dataset_with_length

DatasetDict({
    train: Dataset({
        features: ['id', 'length', 'ner_tags', 'tokens'],
        num_rows: 1350
    })
    validation: Dataset({
        features: ['id', 'length', 'ner_tags', 'tokens'],
        num_rows: 270
    })
    test: Dataset({
        features: ['id', 'length', 'ner_tags', 'tokens'],
        num_rows: 270
    })
})

In [23]:
# Process the dataset - tokenize the context texts (using a tokenizer from the 🤗Transformers library)
from transformers import BertTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained("clue/roberta_chinese_clue_tiny")
tokenizer
# tokenized_dataset = squad_dataset.map(lambda x: tokenizer(x['tokens']), batched=True)

PreTrainedTokenizer(name_or_path='clue/roberta_chinese_clue_tiny', vocab_size=8021, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [24]:
# dataset_with_length = weibo_dataset.map(lambda x: {"length": len(x["tokens"])})


# tokenized_dataset = weibo_dataset.map(lambda x: tokenizer(x['tokens']), batched=True)
# tokenized_dataset
for it  in weibo_dataset['train']:
  # print(it)

  # tag=it['ner_tags']
  for w in it['tokens']:
    print(w.split())
  break

['科0']
['技1']
['全0']
['方1']
['位2']
['资0']
['讯1']
['智0']
['能1']
['，0']
['快0']
['捷1']
['的0']
['汽0']
['车1']
['生0']
['活1']
['需0']
['要1']
['有0']
['三1']
['屏2']
['一0']
['云1']
['爱2']
['你0']


In [25]:
dataset_with_length = weibo_dataset.map(lambda x: {"length": len(x["tokens"])})